In [ ]:
#@title 0: Check GPU

!nvidia-smi

In [2]:
#@title 1: Download training model

!wget -N "https://huggingface.co/uki-kun/jokowi-so-vits-svc-model/resolve/main/G_1528.pth" -P model/jokowi/
!wget -N "https://huggingface.co/uki-kun/jokowi-so-vits-svc-model/resolve/main/config.json" -P model/jokowi/

--2023-05-08 14:11:50--  https://huggingface.co/uki-kun/jokowi-so-vits-svc-model/resolve/main/G_1528.pth
Resolving huggingface.co (huggingface.co)... 18.155.68.38, 18.155.68.116, 18.155.68.121, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/9d/70/9d70fb9f0c6da564646c7771271ed2e759d20c2df41b1fecd64d5e514b6db215/77af5389f8119ddb0c46c00e101fcf957af26c8287467f85e46ef3065a394642?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27G_1528.pth%3B+filename%3D%22G_1528.pth%22%3B&Expires=1683814311&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzlkLzcwLzlkNzBmYjlmMGM2ZGE1NjQ2NDZjNzc3MTI3MWVkMmU3NTlkMjBjMmRmNDFiMWZlY2Q2NGQ1ZTUxNGI2ZGIyMTUvNzdhZjUzODlmODExOWRkYjBjNDZjMDBlMTAxZmNmOTU3YWYyNmM4Mjg3NDY3Zjg1ZTQ2ZWYzMDY1YTM5NDY0Mj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUz

In [3]:
#@title 2: Install library for "downloading YouTube video and converting it to .wav"

!pip install yt_dlp
!pip install ffmpeg
!mkdir yt_audio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 81.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=0aa9be445f4f714c446e8f9ffe715c8c1b96529eb760be67cc10e780b50d41ad
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [4]:
#@title 3: Install Demucs for separating vocal

!python3 -m pip install -U demucs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.8/418.8 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
#@title 4: Install dependencies for AI (may take a long time)

!python -m pip install -U pip wheel
%pip install -U ipython
%pip install -U so-vits-svc-fork

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.7/797.7 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.6 MB/s eta 0:00:00
  Attempting uninstall: ipython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=7.34.0, but you have ipython 8.13.2 which is inco

In [1]:
#@title 5: Download YouTube video and convert it to .wav

from __future__ import unicode_literals
import yt_dlp
import ffmpeg
import sys

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
    "outtmpl": 'yt_audio/audio',
}
def download_from_url(url):
    ydl.download([url])
    # stream = ffmpeg.input('output.m4a')
    # stream = ffmpeg.output(stream, 'output.wav')


with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      url = "https://www.youtube.com/watch?v=K0h71m-7nqg&pp=ygUhZm9yIHdoYXQgaXQncyB3b3J0aCB0aGUgY2FyZGlnYW5z" #@param {type:"string"}
      download_from_url(url)


[youtube] Extracting URL: https://www.youtube.com/watch?v=K0h71m-7nqg&pp=ygUhZm9yIHdoYXQgaXQncyB3b3J0aCB0aGUgY2FyZGlnYW5z
[youtube] K0h71m-7nqg: Downloading webpage
[youtube] K0h71m-7nqg: Downloading android player API JSON
[info] K0h71m-7nqg: Downloading 1 format(s): 251
[dashsegments] Total fragments: 1
[download] Destination: yt_audio/audio
[download] 100% of    4.23MiB in 00:00:00 at 30.67MiB/s              
[ExtractAudio] Destination: yt_audio/audio.wav
Deleting original file yt_audio/audio (pass -k to keep)


In [2]:
#@title 6: Separate vocal from instrument

import subprocess
AUDIO_INPUT = "/content/yt_audio/audio.wav" #@param {type:"string"}

command = f"demucs --two-stems=vocals {AUDIO_INPUT}"
result = subprocess.run(command.split(), stdout=subprocess.PIPE)
print(result.stdout.decode())


Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/separated/htdemucs
Separating track /content/yt_audio/audio.wav



In [6]:
#@title 7: Inference

from IPython.display import Audio

AUDIO = "/content/separated/htdemucs/audio/vocals.wav" #@param {type:"string"}
MODEL = "/content/model/jokowi/G_1528.pth" #@param {type:"string"}
CONFIG = "/content/model/jokowi/config.json" #@param {type:"string"}
#@markdown Change according to the voice tone. 12 = 1 Octave
PITCH_MOD = -12 #@param {type:"integer"}

!svc infer {AUDIO} -c {CONFIG} -m {MODEL} -na -t {PITCH_MOD}
# Try comment this line below if you got Runtime Error
try:
  display(Audio(f"{AUDIO}.out.wav", autoplay=True))
except Exception as e:  print("Error:", str(e))

[14:33:36] WARNING  [14:33:36] auto_predict_f0 = False,          ]8;id=272842;file:///usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/__main__.py\__main__.py]8;;\:]8;id=367376;file:///usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/__main__.py#246\246]8;;\
                    transpose = -12. If you want to change the                  
                    pitch, please set transpose.Generally                       
                    transpose = 0 does not work because your                    
                    voice pitch and target voice pitch are                      
                    different.                                                  
[14:33:39] WARNING  [14:33:39]                                   ]8;id=572207;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=428981;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                    /usr/local/lib/python3.10/dist-packages/so_v                
                    its_sv

In [7]:
#@title 8: Combine Vocal and Instrument (Song Cover)
!pip install pydub
from pydub import AudioSegment

FILENAME = "for-what-its-worth" #@param {type: "string"}
VOCAL = "/content/separated/htdemucs/audio/vocals.out.wav" #@param {type:"string"}
INSTRUMENT = "/content/separated/htdemucs/audio/no_vocals.wav" #@param {type:"string"}

sound1 = AudioSegment.from_file(VOCAL)
sound2 = AudioSegment.from_file(INSTRUMENT)

combined = sound1.overlay(sound2)

combined.export(f"/content/{FILENAME}", format='wav')
try:
  display(Audio(f"/content/FinalCover.wav", autoplay=True))
except Exception as e:  print("Error:", str(e))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title Additional: Install module for audio recording

!pip install ffmpeg-python

In [1]:
#@title Record audio

"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
audio, sr = get_audio()

In [8]:
from scipy.io import wavfile
FILENAME = "recording.wav" #@param {type:"string"}
wavfile.write(FILENAME, sr, audio)